In [20]:
import requests
import pandas as pd


In [24]:
# The following code curtesy of James Briggs: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

## Fetch an OAUTH Token. It is only valid for 2 hours and will need to be re-requested when it expires.

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('fOIoNDAQnOFBiQ5RM9CEuA', 'Rb9whMn_xOwNVra9usbbEPYkVtMqvw')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'flatest_iron',
        'password': 'FlatironPassword19!'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [25]:
# Get the top posts of all time from r/python
response = requests.get("https://oauth.reddit.com/r/python/top/?t=all", headers=headers)

In [26]:
# Build one part of the dataframe 

python_df = pd.DataFrame()

#We can add to this if need be; there is a lot of data that isn't being captured here
for post in response.json()['data']['children']:
    python_df = python_df.append({
        'subreddit' : post['data']['subreddit'],
        'title' : post['data']['title'],
        'text' : post['data']['selftext'],
        'ups' : post['data']['ups'],
        'downs' : post['data']['downs'] 
    }, ignore_index = True)



In [27]:
python_df

,downs,subreddit,text,title,ups
0,0.0,Python,,Lad wrote a Python script to download Alexa vo...,12344.0
1,0.0,Python,"9777 upvotes,\n\n967 downvotes\n\nand 452 comm...",This post has:,9239.0
2,0.0,Python,,I redesign the Python logo to make it more modern,7861.0
3,0.0,Python,,Automate the boring stuff with python - tinder,6726.0
4,0.0,Python,,Just finished programming and building my own ...,6607.0
5,0.0,Python,,"I'm excited to share my first published book, ...",6502.0
6,0.0,Python,,Drawing Mona Lisa with 256 circles using evolu...,5720.0
7,0.0,Python,,I made a simulation using Python in which a ne...,5689.0
8,0.0,Python,,"Thanks to everyone’s advice, my mouse drawing ...",5544.0
9,0.0,Python,,Debugging Cheat Sheet,5448.0


In [ ]:
# Create a list of a subreddits that we want to fetch the top posts for

# There might be a better way to do this, but for right now, I'm just gonna do it manually
# My reasoning for this is that I can grab some less popular reddits that some of the listings
subreddits = [
    'BlackPeopleTwitter',
    'WhitePeopleTwitter',
    'politics',
    'news',
    'Jokes',
    'dankmemes',
    'antiwork',
    'funny',
    'gaming',
    'technology',
    'dataisbeautiful',
    'MurderedByWords',
    'mildlyinteresting',
    'wallstreetbets',
    'shitposting',
    'pcmasterrace',
    'niceguys',
    'teenagers',
    'PublicFreakout',
    'SelfAwarewolves',
    'dndmemes',
    'memes',
    'meirl',
    'PolitcalCompassMemes',
    'AskReddit',
    'onejoke',
    'Feminism',
    'AreTheStraightsOK',
    'LeopardsAteMyFace',
    'antifeminists',
    'MensRights',
    'ProudMaleFeminists',
    'TwoXChromosomes',
    'WhereAreTheFeminists',
    'FeminismUncensored',
    'ShitRedditSays',
    'SRSsucks',
    'Conservative',
    'JordanPeterson',
    'Anarchism',
    'TopMindsOfReddit',
    'conspiracy',
    'tucker_carlson',
    'TheTrumpZone',
    'ToiletPaperUSA',
    'LouderWithCrowder',
    'Libertarian',
    'AskTheDonald',
    'walkaway',
    'esist',
    'ConservativesOnly',
    'MarchAgainstNazis',
    'GamingCircleJerk',
    'IncelsCircleJerk',
    'BlackPilledScience',
    'Blubber_Whaling',
    'memegender',
]